In [2]:
# For reproducibility
np.random.seed(42)
import random
random.seed(42)


In [10]:
# Install dependencies
!pip install adversarial-robustness-toolbox
!pip install ai-privacy-toolkit

# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from art.estimators.classification.scikitlearn import ScikitlearnDecisionTreeClassifier
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
from apt.utils.datasets import ArrayDataset
from apt.anonymization import Anonymize

# Download and load dataset
!mkdir -p ../datasets
!wget -P ../datasets https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

data = pd.read_csv("../datasets/adult.data", header=None, skipinitialspace=True)

# Numerical features only
num_features = [0, 4, 10, 11, 12]
X = data.iloc[:, num_features].values
y = data.iloc[:, 14].values

y = np.array([label.strip() for label in y])
y = np.where(y == "<=50K", 0, 1)

# Split data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Train Decision Tree
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
art_classifier = ScikitlearnDecisionTreeClassifier(model)
print('Base model accuracy:', model.score(x_test, y_test))

# Membership Inference Attack on Base Model
bb_attack = MembershipInferenceBlackBox(art_classifier, attack_model_type='rf')
attack_train_ratio = 0.5
attack_train_size = int(len(x_train) * attack_train_ratio)
attack_test_size = int(len(x_test) * attack_train_ratio)

bb_attack.fit(x_train[:attack_train_size], y_train[:attack_train_size],
              x_test[:attack_test_size], y_test[:attack_test_size])

inferred_train_bb = bb_attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
inferred_test_bb = bb_attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])

train_acc = np.sum(inferred_train_bb) / len(inferred_train_bb)
test_acc = 1 - (np.sum(inferred_test_bb) / len(inferred_test_bb))
acc = (train_acc * len(inferred_train_bb) + test_acc * len(inferred_test_bb)) / (len(inferred_train_bb) + len(inferred_test_bb))
print("Attack accuracy on base model:", acc)

# Anonymize data
x_train_predictions = np.array([np.argmax(arr) for arr in art_classifier.predict(x_train)])
QI = [0, 1, 2, 4]
anonymizer = Anonymize(100, QI)
anon = anonymizer.anonymize(ArrayDataset(x_train, x_train_predictions))
print(anon)

# number of distinct rows in original data
len(np.unique(x_train, axis=0))

# number of distinct rows in anonymized data
len(np.unique(anon, axis=0))

# Train on Anonymized Data
anon_model = DecisionTreeClassifier()
anon_model.fit(anon, y_train)
anon_art_classifier = ScikitlearnDecisionTreeClassifier(anon_model)
print('Anonymized model accuracy:', anon_model.score(x_test, y_test))

# Membership Inference Attack on Anonymized Model
anon_bb_attack = MembershipInferenceBlackBox(anon_art_classifier, attack_model_type='rf')
anon_bb_attack.fit(x_train[:attack_train_size], y_train[:attack_train_size],
                   x_test[:attack_test_size], y_test[:attack_test_size])

anon_inferred_train_bb = anon_bb_attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
anon_inferred_test_bb = anon_bb_attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])

anon_train_acc = np.sum(anon_inferred_train_bb) / len(anon_inferred_train_bb)
anon_test_acc = 1 - (np.sum(anon_inferred_test_bb) / len(anon_inferred_test_bb))
anon_acc = (anon_train_acc * len(anon_inferred_train_bb) + anon_test_acc * len(anon_inferred_test_bb)) / (len(anon_inferred_train_bb) + len(anon_inferred_test_bb))
print("Attack accuracy on anonymized model:", anon_acc)

def calc_precision_recall(predicted, actual, positive_value=1):
  score = 0 # both predicted and actual are positive
  num_positive_predicted = 0 # predicted positive
  num_positive_actual = 0 # actual positive
  for i in range(len(predicted)):
    if predicted[i] == positive_value:
      num_positive_predicted += 1
    if actual[i] == positive_value:
      num_positive_actual += 1
    if predicted[i] == actual[i]:
      if predicted[i] == positive_value:
        score += 1
  if num_positive_predicted == 0:
     precision = 1
  else:
    precision = score / num_positive_predicted # the fraction of predicted “Yes” responses that are correct
  if num_positive_actual == 0:
    recall = 1
  else:
    recall = score / num_positive_actual # the fraction of “Yes” responses that are predicted correctly
  return precision, recall
# regular
print('without anonymization:', calc_precision_recall(np.concatenate((inferred_train_bb, inferred_test_bb)),
np.concatenate((np.ones(len(inferred_train_bb)), np.zeros(len(inferred_test_bb))))))
# anon
print('with anonymization:', calc_precision_recall(np.concatenate((anon_inferred_train_bb, anon_inferred_test_bb)),
np.concatenate((np.ones(len(anon_inferred_train_bb)), np.zeros(len(anon_inferred_test_bb))))))


--2025-02-23 22:48:22--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘../datasets/adult.data.7’

adult.data.7            [          <=>       ]   3.79M  99.8KB/s    in 45s     

2025-02-23 22:49:08 (86.8 KB/s) - ‘../datasets/adult.data.7’ saved [3974305]

Base model accuracy: 0.8172884999232305
Attack accuracy on base model: 0.8143234445058657
[[34 13  0  0 45]
 [42 13  0  0 45]
 [20  9  0  0 24]
 ...
 [50  9  0  0 50]
 [25 13  0  0 50]
 [30  6  0  0 40]]
Anonymized model accuracy: 0.8340242591739597
Attack accuracy on anonymized model: 0.7978011178674529
without anonymization: (0.8124726613759921, 0.9983108108108109)
with anonymization: (0.7998521074685728, 0.9966216216216216)
